In [19]:
import requests
import sqlite3
import time
import json

# API base URL
BASE_URL = "https://api.www.sbir.gov/public/api/solicitations"

# Database setup
DB_NAME = "solicitations.db"

# Function to create the database tables
def setup_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Create solicitations table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS solicitations (
            solicitation_id INTEGER PRIMARY KEY,
            solicitation_title TEXT,
            solicitation_number TEXT UNIQUE,
            program TEXT,
            phase TEXT,
            agency TEXT,
            branch TEXT,
            solicitation_year INTEGER,
            release_date TEXT,
            open_date TEXT,
            close_date TEXT,
            application_due_dates TEXT,
            solicitation_agency_url TEXT,
            current_status TEXT
        )
    """)
    
    # Create topics table (Only if more than one exists per solicitation)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS topics (
            topic_number TEXT PRIMARY KEY,
            solicitation_id INTEGER,
            topic_title TEXT,
            branch TEXT,
            topic_description TEXT,
            sbir_topic_link TEXT,
            topic_open_date TEXT,
            topic_closed_date TEXT,
            FOREIGN KEY (solicitation_id) REFERENCES solicitations(solicitation_id)
        )
    """)

    # Create subtopics table linked to topics
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS subtopics (
            subtopic_number TEXT PRIMARY KEY,
            topic_number TEXT,
            subtopic_title TEXT,
            branch TEXT,
            subtopic_description TEXT,
            FOREIGN KEY (topic_number) REFERENCES topics(topic_number)
        )
    """)

    conn.commit()
    conn.close()

# Function to fetch data from API
def fetch_data(page):
    params = {
        "rows": 50,
        "start": page * 50,
        #"open": 1,
        #"agency": "HHS",
        #"keyword": "xyz"
        }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching page {page}: {response.status_code}")
        return None

# Function to insert data into SQLite database
def insert_data(data):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    for item in data:
        # Insert into solicitations table
        cursor.execute("""
            INSERT OR REPLACE INTO solicitations (
                solicitation_id, solicitation_title, solicitation_number, program, phase, agency, branch,
                solicitation_year, release_date, open_date, close_date, application_due_dates,
                solicitation_agency_url, current_status
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            item.get("solicitation_id"),
            item.get("solicitation_title"),
            item.get("solicitation_number"),
            item.get("program"),
            item.get("phase"),
            item.get("agency"),
            item.get("branch"),
            item.get("solicitation_year"),
            item.get("release_date"),
            item.get("open_date"),
            item.get("close_date"),
            json.dumps(item.get("application_due_date")),  # Store as JSON string
            item.get("solicitation_agency_url"),
            item.get("current_status")
        ))

        # Insert into topics table only if there are multiple topics
        topics = item.get("solicitation_topics", [])
        if len(topics) > 1:
            for topic in topics:
                cursor.execute("""
                    INSERT OR REPLACE INTO topics (
                        topic_number, solicitation_id, topic_title, branch, topic_description,
                        sbir_topic_link, topic_open_date, topic_closed_date
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    topic.get("topic_number"),
                    item.get("solicitation_id"),
                    topic.get("topic_title"),
                    topic.get("branch"),
                    topic.get("topic_description"),
                    topic.get("sbir_topic_link"),  # Now stored in topics
                    topic.get("topic_open_date"),
                    topic.get("topic_closed_date")
                ))

                # Insert subtopics only if they exist
                subtopics = topic.get("subtopics", [])
                for subtopic in subtopics:
                    cursor.execute("""
                        INSERT OR REPLACE INTO subtopics (
                            subtopic_number, topic_number, subtopic_title, branch, subtopic_description
                        ) VALUES (?, ?, ?, ?, ?)
                    """, (
                        subtopic.get("subtopic_number"),
                        topic.get("topic_number"),
                        subtopic.get("subtopic_title"),
                        subtopic.get("branch"),
                        subtopic.get("subtopic_description")
                    ))

    conn.commit()
    conn.close()

# Main execution
def main():
    setup_database()
    
    for page in range(2):  # Fetch only two pages for testing
        print(f"Fetching page {page + 1}...")
        data = fetch_data(page)
        if data:
            insert_data(data)
        time.sleep(1)  # Avoid excessive requests

    print("Data successfully stored in SQLite database.")

if __name__ == "__main__":
    main()


Fetching page 1...
Fetching page 2...
Data successfully stored in SQLite database.


In [21]:
def fetch_solicitations():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM solicitations LIMIT 10")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

def fetch_topics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM topics LIMIT 10")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

def fetch_subtopics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM subtopics LIMIT 10")
    rows = cursor.fetchall()

    for row in rows:
        print(row)
    
    conn.close()

def count_entries():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Count entries in solicitations table
    cursor.execute("SELECT COUNT(*) FROM solicitations")
    solicitation_count = cursor.fetchone()[0]
    
    # Count entries in topics table
    cursor.execute("SELECT COUNT(*) FROM topics")
    topic_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM subtopics")
    subtopic_count = cursor.fetchone()[0]
    
    conn.close()
    
    return {
        "solicitations": solicitation_count,
        "topics": topic_count,
        "subtopics": subtopic_count
    }

if __name__ == "__main__":
    print("Solicitations:")
    fetch_solicitations()
    
    print("\nTopics:")
    fetch_topics()

    print("\nSubtopics:")
    fetch_subtopics()

    counts = count_entries()
    print(f"Number of solicitations: {counts['solicitations']}")
    print(f"Number of topics: {counts['topics']}")
    print(f"Number of subtopics: {counts['subtopics']}")

Solicitations:
(947, 'Novel Tools and Devices for Animal Research Facilities and to Support Care of Animal Models (R41/R42 Clinical Trial Not Allowed)', 'PAR-21-226', 'STTR', 'BOTH', 'HHS', 'NIH', 2021, '2021/04/30', '2021/08/05', '2024/09/06', '["2021/09/06", "2022/01/05", "2022/04/05", "2022/09/05", "2023/01/05", "2023/04/05", "2023/09/05", "2024/01/05", "2024/04/05"]', 'https://grants.nih.gov/grants/guide/PA-files/PAR-21-226.html', 'closed')
(954, 'Novel Tools and Devices for Animal Research Facilities and to Support Care of Animal Models (R43/R44 Clinical Trial Not Allowed)', 'PAR-21-225', 'SBIR', 'BOTH', 'HHS', 'NIH', 2021, '2021/04/30', '2021/08/06', '2024/09/06', '["2021/09/05", "2022/01/05", "2022/04/05", "2022/09/05", "2023/01/05", "2023/04/05", "2023/09/05", "2024/01/05", "2024/04/05", "2024/09/05"]', 'https://grants.nih.gov/grants/guide/pa-files/PAR-21-225.html', 'closed')
(983, 'Blueprint Medtech: Small Business Translator (U44 - Clinical Trial Optional)', 'PAR-21-282', 'SB